In [1]:
#%run <path of the notebook that has to be run>

In [1]:
from cobra.io import read_sbml_model, write_sbml_model

In [2]:
model = read_sbml_model('model_yeast8_rhb.xml')

In [3]:
print(model.objective),print("Growth rate:", model.optimize().objective_value)

Maximize
1.0*GROWTH - 1.0*GROWTH_reverse_c845e
Growth rate: 0.8637367930478724


(None, None)

### Function

In [4]:
def test_knockout(model, gene_id):
    """Input 1: a SBML BiGG compliant GSMM.
    
    Input 2: ID of the gene of interest, as type 
    string or a list of several genes.
    """
    
    print("---Before knockout---")
    print("Growth rate:", model.optimize().objective_value)
    copy_of_model1 = model.copy()
    with copy_of_model1:
        copy_of_model1.objective = copy_of_model1.reactions.EX_rHb # Change in recombinant hemoglobin   
        print("Rate of hemoglobin production:", copy_of_model1.reactions.rHb.flux," [mmol/gDW/h]")
        print("Yield for hemoglobin on sugar", copy_of_model1.reactions.rHb.flux / -1*(copy_of_model1.reactions.EX_glc__D_e.flux),"[mmol-rHb / mmol-glucose]")
        print("--------------------")
    
        
    copy_of_model2 = model.copy()
    with copy_of_model2 as test_mutant: 
        print("---After knockout---")
        
        if type(gene_id) is list: # If you want to knock-out multiple genes
            for i in range (0,len(gene_id)):
                gene = test_mutant.genes.get_by_id(gene_id[i]) # gene to knock out
                gene.knock_out() # knocks out the gene
        else: # if only one gene should be knocked-out
            gene = test_mutant.genes.get_by_id(gene_id) # gene to knock out
            gene.knock_out() # knocks out the gene
        
        test_mutant.objective = test_mutant.reactions.rHb # Change in recombinant hemoglobin
        print("Growth rate:", test_mutant.optimize().objective_value)#+" from the old growth rate, or a change of "+str(rate_change/old_rate*100)[:5]+"%.")     
        print("Rate of hemoglobin production:", test_mutant.reactions.EX_rHb.flux," [mmol/gDW/h]")
        print("Yield for hemoglobin on sugar", test_mutant.reactions.EX_rHb.flux / -1*(test_mutant.reactions.EX_glc__D_e.flux),"[mmol-rHb / mmol-glucose]")
        print("--------------------")
        #print("Before the knockout, the theoretical maximum yield of rHb on glucose [mmol-rHb / mmol-glucose] was: "+str(old_rhb_yield)[:9]+". Now the yield is: "+str(new_rhb_yield)[:9]+" which is a change of "+str(yield_change)+".") #+" and the rate is: "+str(new_rhb_production)[:6]

Knocking out a single gene

#### Let's kill COX10 (YPL172C)

In [5]:
test_knockout(model, 'YPL172C')

---Before knockout---
Growth rate: 0.8637367930478724
Rate of hemoglobin production: 0.0  [mmol/gDW/h]
Yield for hemoglobin on sugar 0.0 [mmol-rHb / mmol-glucose]
--------------------
---After knockout---
Growth rate: 0.01633390069511925
Rate of hemoglobin production: 0.01633390069511925  [mmol/gDW/h]
Yield for hemoglobin on sugar 0.1633390069511925 [mmol-rHb / mmol-glucose]
--------------------


#### Now let's try for CYC3.

In [6]:
test_knockout(model, 'YAL039C')

---Before knockout---
Growth rate: 0.8637367930478724
Rate of hemoglobin production: 0.0  [mmol/gDW/h]
Yield for hemoglobin on sugar 0.0 [mmol-rHb / mmol-glucose]
--------------------
---After knockout---
Growth rate: 0.016333900695119228
Rate of hemoglobin production: 0.016333900695119228  [mmol/gDW/h]
Yield for hemoglobin on sugar 0.16333900695119227 [mmol-rHb / mmol-glucose]
--------------------


#### Now let's try for YFH1

In [7]:
test_knockout(model, 'YDL120W')

---Before knockout---
Growth rate: 0.8637367930478724
Rate of hemoglobin production: 0.0  [mmol/gDW/h]
Yield for hemoglobin on sugar 0.0 [mmol-rHb / mmol-glucose]
--------------------
---After knockout---
Growth rate: 0.016333900695119228
Rate of hemoglobin production: 0.016333900695119228  [mmol/gDW/h]
Yield for hemoglobin on sugar 0.16333900695119227 [mmol-rHb / mmol-glucose]
--------------------


#### Let's try them all

In [8]:
test_knockout(model, ['YDL120W','YAL039C','YPL172C'])

---Before knockout---
Growth rate: 0.8637367930478724
Rate of hemoglobin production: 0.0  [mmol/gDW/h]
Yield for hemoglobin on sugar 0.0 [mmol-rHb / mmol-glucose]
--------------------
---After knockout---
Growth rate: 0.01633390069511925
Rate of hemoglobin production: 0.01633390069511925  [mmol/gDW/h]
Yield for hemoglobin on sugar 0.1633390069511925 [mmol-rHb / mmol-glucose]
--------------------


## GeckoPy